##Connect to Gdrive because I uploaded files in there

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.chdir("/content/drive/My Drive/Mobility")


### Loaded packages (some of them are extra, will clean later)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sklearn 
from datetime import datetime
from sklearn import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# !pip3 install bayesian-optimization
from bayes_opt import BayesianOptimization


from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier ,ExtraTreesClassifier,AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm
from lightgbm import LGBMClassifier
# !pip3 install catboost
from catboost import CatBoostClassifier
import operator

### Read train test files and EDA (Need to do more EDA)

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
print(train.isna().sum())

In [ ]:
print(test.isna().sum())

In [ ]:
train.dtypes

In [ ]:
train.select_dtypes(include=['object'])

In [ ]:
sns.boxplot(x='Customer_Since_Months',data=train,color='red')

In [ ]:
sns.boxplot(x='Life_Style_Index',data=train,color='red')

In [ ]:
sns.boxplot(x='Var1',data=train,color='red')

In [ ]:
print(train.isna().sum())

In [ ]:
train.head()

In [ ]:
df_num_corr = train.corr()['Surge_Pricing_Type'][:-1]
features_list = df_num_corr[df_num_corr > 0.1].sort_values(ascending=False)
features_list

In [ ]:
train_num = train.select_dtypes(include = ['float64', 'int64'])
individual_features_df = []
for i in range(0, len(train_num.columns) - 1):
    tmpDf = train_num[[train_num.columns[i], 'Surge_Pricing_Type']]
    tmpDf = tmpDf[tmpDf[train_num.columns[i]] != 0]
    individual_features_df.append(tmpDf)

all_correlations = {feature.columns[0]: feature.corr()['Surge_Pricing_Type'][0] for feature in individual_features_df}
all_correlations = sorted(all_correlations.items(), key=operator.itemgetter(1))
for (key, value) in all_correlations:
    print("{:>20}: {:>5}".format(key, value))

In [ ]:
train['Surge_Pricing_Type'].describe()

In [ ]:
train.hist(figsize=(20, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:
corr = train.drop('Surge_Pricing_Type', axis=1).corr() 
plt.figure(figsize=(12, 10))

sns.heatmap(corr, 
             vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

In [ ]:
train_categ_columns = list(train.select_dtypes(include=['object']))
train_categ_columns.append('Surge_Pricing_Type')
train_categ = train[train_categ_columns]
train_categ.head()

In [ ]:
plt.figure(figsize = (10, 6));
ax = sns.boxplot(x='Type_of_Cab', y='Surge_Pricing_Type', data=train_categ);
plt.setp(ax.artists, alpha=.5, linewidth=2, edgecolor="k");
plt.xticks(rotation=45);

In [ ]:
plt.figure(figsize = (10, 6));
ax = sns.boxplot(x='Confidence_Life_Style_Index', y='Surge_Pricing_Type', data=train_categ);
plt.setp(ax.artists, alpha=.5, linewidth=2, edgecolor="k");
plt.xticks(rotation=45);

In [ ]:
plt.figure(figsize = (10, 6));
ax = sns.boxplot(x='Destination_Type', y='Surge_Pricing_Type', data=train_categ);
plt.setp(ax.artists, alpha=.5, linewidth=2, edgecolor="k");
plt.xticks(rotation=45);

In [ ]:
plt.figure(figsize = (10, 6));
ax = sns.boxplot(x='Gender', y='Surge_Pricing_Type', data=train_categ);
plt.setp(ax.artists, alpha=.5, linewidth=2, edgecolor="k");
plt.xticks(rotation=45);

In [ ]:
#disabled, uses high cpu in my laptop (Progha)
#fig, axes = plt.subplots(round(len(train_categ.columns) / 3), 3, figsize=(15, 15))

#for i, ax in enumerate(fig.axes):
#    if i < len(train_categ.columns):
#        ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)
#        sns.countplot(x=train_categ.columns[i], alpha=0.7, data=train_categ, ax=ax)

#fig.tight_layout()

## Treatment of NA Values (Can try more things)

### NA Values in Train and Test data

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

### Value counts for Object Type Columns (categorical data)

In [ ]:
objtype_columns = list(train.select_dtypes(include=['object']).columns)
# No missing values for Trip_ID
if 'Trip_ID' in objtype_columns:
    objtype_columns.remove('Trip_ID')
# pricing should not depend on Gender    
if 'Gender' in objtype_columns:
    objtype_columns.remove('Gender')

for objtype_col in objtype_columns:
    print('Train data, Value counts for ' + str(objtype_col))
    display(train[objtype_col].value_counts())
    print('Test data, Value counts for ' + str(objtype_col))
    display(test[objtype_col].value_counts())    

### fillna for Object Type Columns (categorical data)

In [ ]:
# We are treating missing values as a feature, so updating them with the value "Unknown"
# If we do not want to fill missing values for some column (e.g. Gender), we need to update objtype_columns.
use_most_common_values_for_cat_data_fillna = True

# As Pavani suggested, we can also use most common values for filling na values (for categorical data)
# In this case, set use_most_common_values_for_cat_data_fillna to True

if use_most_common_values_for_cat_data_fillna == False:
    for objtype_col in objtype_columns:
        train[objtype_col] = train[objtype_col].fillna("Unknown")
        test[objtype_col] = test[objtype_col].fillna("Unknown")
else:
    common_value_map = {'Type_of_Cab': "B", 'Confidence_Life_Style_Index': "B"}
    for key, value in common_value_map.items():
        train[key].fillna(value, inplace=True)
        test[key].fillna(value, inplace=True)

#### Update NA values (continuous) with mean/median

In [ ]:
for col in ['Customer_Since_Months','Var1']:
    train[col].fillna(train[col].median(), inplace = True)
    test[col].fillna(test[col].median(), inplace = True)
    
for col in ['Life_Style_Index']:
    train[col].fillna(train[col].mean(), inplace = True)
    test[col].fillna(test[col].mean(), inplace = True)

#### Convert datatypes (minor transformation)

In [ ]:
for col in ['Customer_Since_Months', 'Var1']:
    train[col] = train[col].astype('int')
    test[col] = test[col].astype('int')

In [ ]:
train.head()

In [ ]:
test.head()

### Label Encoding (We can decide if necessary or not)

In [ ]:
train.Type_of_Cab.value_counts()

In [ ]:
le = LabelEncoder()
trained_le = le.fit(train["Type_of_Cab"])
train["Type_of_Cab"] = trained_le.transform(train["Type_of_Cab"])
test["Type_of_Cab"] = trained_le.transform(test["Type_of_Cab"])
train.head(10)

In [ ]:
train.Confidence_Life_Style_Index.value_counts()

In [ ]:
le = LabelEncoder()
trained_le = le.fit(train["Confidence_Life_Style_Index"])
train["Confidence_Life_Style_Index"] = trained_le.transform(train["Confidence_Life_Style_Index"])
test["Confidence_Life_Style_Index"] = trained_le.transform(test["Confidence_Life_Style_Index"])
train.head(10)

In [ ]:
train.Destination_Type.value_counts()

In [ ]:
le = LabelEncoder()
trained_le = le.fit(train["Destination_Type"])
train["Destination_Type"] = trained_le.transform(train["Destination_Type"])
test["Destination_Type"] = trained_le.transform(test["Destination_Type"])
train.head(10)

In [ ]:
le = LabelEncoder()
trained_le = le.fit(train["Gender"])
train["Gender"] = trained_le.transform(train["Gender"])
test["Gender"] = trained_le.transform(test["Gender"])
train.head(10)

In [ ]:
train.dtypes

In [ ]:
train.head()

In [ ]:
df_num_corr = train.corr()['Surge_Pricing_Type'][:-1]
features_list = df_num_corr[abs(df_num_corr) > 0.1].sort_values(ascending=False)
features_list